## 2.1 MLP Multi-Class Classifier

### Functions

In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import cv2
from sklearn.preprocessing import OneHotEncoder
from tqdm import tqdm
import wandb
from sklearn.preprocessing import StandardScaler
wandb.login(key="cb255956b462ea96bcedb0051bc14581a02f25c3")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Admin\_netrc
wandb: Currently logged in as: ketakishetye0619 (ketakishetye0619-iiit-hyderabad) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [2]:
class MLP():
    def __init__(self,input_dim,hidden_dim,output_dim,activation):
        self.in_hidden_weights=np.random.randn(input_dim, hidden_dim)*np.sqrt(2/input_dim)
        self.out_hidden_weights=np.random.randn(hidden_dim,output_dim)*np.sqrt(2/hidden_dim)
        self.bias_hidden = np.zeros((1, hidden_dim))
        self.bias_output=np.zeros((1,output_dim))
        self.activation=activation
        
    def apply_activation(self,x):
        if self.activation=='relu':
            activation = np.maximum(0, x)  
        elif self.activation=='sigmoid':
            activation = 1 / (1 + np.exp(-x))
        elif self.activation=='tanh':
            activation = np.tanh(x)  
            
        return activation
    
    def apply_derivative(self,x):
        if self.activation=='relu':
            derivative=np.where(x > 0, 1, 0)
        elif self.activation=='sigmoid':
            activation = 1 / (1 + np.exp(-x))
            derivative= activation * (1 - activation)  
        elif self.activation=='tanh':
            activation = np.tanh(x)  
            derivative = 1 - activation ** 2  
            
        return derivative
    
    def softmax(self,x):
        exponent_of_x=np.exp(x-np.max(x, axis=1, keepdims=True)) # the largest value in x becomes 0, making all exponentials ≤ 1 => to prevent large exponents
        return exponent_of_x / np.sum(exponent_of_x,axis=1, keepdims=True)
    
    def forward(self,x):
        self.hidden_layer_input=np.dot(x,self.in_hidden_weights)+self.bias_hidden
        self.hidden_layer_output=self.apply_activation(self.hidden_layer_input)
        self.final_input=np.dot(self.hidden_layer_output,self.out_hidden_weights)+self.bias_output
        self.final_output=self.softmax(self.final_input)
        return self.final_output
    
    def backward_propogation(self,x,y_true,y_pred,learning_rate):
        batch_size=x.shape[0]
        output_layer_error=(y_pred-y_true)/batch_size
        hidden_layer_error=(np.dot(output_layer_error,self.out_hidden_weights.T))*self.apply_derivative(self.hidden_layer_input) # total contribution of each hidden neuron to the overall output error and multiplied by derivative as scales the hidden error based on how much each hidden neuron is "activated."

        self.out_hidden_weights=self.out_hidden_weights-learning_rate*(np.dot(self.hidden_layer_output.T,output_layer_error))
        self.bias_output=self.bias_output-learning_rate*(np.sum(output_layer_error,axis=0,keepdims=True))
        
        self.in_hidden_weights=self.in_hidden_weights-learning_rate*(np.dot(x.T,hidden_layer_error))
        self.bias_hidden=self.bias_hidden-learning_rate*(np.sum(hidden_layer_error,axis=0,keepdims=True))
        
    def cross_entropy(self,y_true,y_pred,x):
        loss=-np.sum(y_true*np.log(y_pred))/x.shape[0]
        return loss
        
    def calculate_train_accuracy(self, y_true, y_pred):
        y_true_indices = np.argmax(y_true, axis=-1)
        y_pred_indices = np.argmax(y_pred, axis=-1)
        accuracy = np.mean(y_true_indices == y_pred_indices)
        return accuracy*100
    
    def calculate_test_accuracy(self, y_true, y_pred):
        y_true_indices = np.argmax(y_true, axis=-1)
        accuracy = np.mean(y_true_indices == y_pred)
        return accuracy*100
        
    def predict(self, x):
        return np.argmax(self.forward(x), axis=-1)
    
    def train(self,x_train,y_true,num_epochs,learning_rate,x_test,y_test,fold_index,batch_size):
        
        total_num_train_samples=x_train.shape[0]
        if batch_size is None:
            batch_size=total_num_train_samples
            
        epoch_iterator = tqdm(
            range(num_epochs),
            desc=f"Fold {fold_index+1}/10",
            unit="epoch",
            ncols=100,
            leave=True
        )

        for epoch in epoch_iterator:
            
            indices=np.random.permutation(total_num_train_samples)
            x_shuffled_train = x_train[indices]
            y_shuffled_train= y_true[indices]
            
            epoch_loss = 0
            epoch_accuracy = 0
            num_batches = 0
        
            for i in range(0, total_num_train_samples, batch_size):
                end = min(i + batch_size, total_num_train_samples)
                x_batch = x_shuffled_train[i:end]
                y_batch = y_shuffled_train[i:end]
                
                y_pred=self.forward(x_batch)
                self.backward_propogation(x_batch,y_batch,y_pred,learning_rate)
                
                loss=self.cross_entropy(y_batch,y_pred,x_batch)
                epoch_loss=epoch_loss+loss
                
                accuracy=self.calculate_train_accuracy(y_batch,y_pred)
                epoch_accuracy=epoch_accuracy+accuracy
                
                num_batches=num_batches+1
                
            y_pred_test=self.predict(x_test)
            test_accuracy=self.calculate_test_accuracy(y_test,y_pred_test)
            
            if (epoch+1)%10==0:
                print(f"Epoch: {epoch},Loss:{epoch_loss/num_batches}")
                print(f"Train Accuracy:{epoch_accuracy/num_batches},Test Accuracy:{test_accuracy}")
            elif epoch==0:
                print(f"Epoch: {epoch},Loss:{epoch_loss/num_batches}")
                print(f"Train Accuracy:{epoch_accuracy/num_batches},Test Accuracy:{test_accuracy}")
            
            wandb.log({
                "fold index": fold_index + 1,
                "epoch": epoch,
                "train loss": epoch_loss/num_batches,
                "train_accuracy": epoch_accuracy/num_batches,
                "test_accuracy": test_accuracy
            })

In [3]:
np.random.seed(42)
input_dim=1024
hidden_dim=512
learning_rate=0.01
num_epochs=50
activation='relu'
batch_size=None

In [3]:
folder="Symbols"
def load_data(fold_path):
    df=pd.read_csv(fold_path)
    images=[]
    labels = df['symbol_id'].values.reshape(-1, 1)
    for image_path in df['path']:
        actual_image_path = os.path.join(*image_path.split("/")[-2:])
        image = cv2.imread(os.path.join(folder, actual_image_path), cv2.IMREAD_GRAYSCALE) #can convert to grayscale as no colours in it
        image = cv2.resize(image, (32, 32)).flatten() / 255.0
        images.append(image)
    return np.array(images), labels

In [5]:
wandb.init(project="SMAI-A2", name="2.1-Relu-BGD")

fold_dirs = [f'Symbols/classification-task/fold-{i}' for i in range(1, 11)]

test_accuracies=[]
for fold_index,fold in enumerate(fold_dirs):
    print("Loading Data")
    train_path = os.path.join(fold, 'train.csv')
    test_path = os.path.join(fold, 'test.csv')
    x_train,y_train=load_data(train_path)
    x_test,y_test=load_data(test_path)
    
    encoder = OneHotEncoder(sparse_output=False)
    y_train = encoder.fit_transform(y_train)
    encoder = OneHotEncoder(sparse_output=False)
    y_test = encoder.fit_transform(y_test)
    
    output_dim=y_train.shape[1]
    
    print("Initialising model")
    model = MLP(input_dim, hidden_dim, output_dim,activation)   
    model.train(x_train,y_train,num_epochs,learning_rate,x_test,y_test,fold_index,batch_size)
    y_pred_test=model.predict(x_test)
    test_accuracy=model.calculate_test_accuracy(y_test,y_pred_test)
    test_accuracies.append(test_accuracy)
    
print(f"Mean test accuracy:{np.mean(test_accuracies)} ,Standard deviation:{np.std(test_accuracies)}")
wandb.finish()

Loading Data
Initialising model


Fold 1/10:   2%|█                                                 | 1/50 [00:10<08:11, 10.04s/epoch]

Epoch: 0,Loss:6.742725595315644
Train Accuracy:0.08926150977578831,Test Accuracy:0.10004708097928437


Fold 1/10:  20%|█████████▊                                       | 10/50 [01:24<05:32,  8.31s/epoch]

Epoch: 9,Loss:6.104334657521007
Train Accuracy:0.6552456013911572,Test Accuracy:0.759180790960452


...

Epoch: 39,Loss:5.5951017189344645
Train Accuracy:3.19650067294751,Test Accuracy:3.3371346257727628


Fold 10/10: 100%|████████████████████████████████████████████████| 50/50 [06:07<00:00,  7.36s/epoch]

Epoch: 49,Loss:5.533541056405469
Train Accuracy:3.591032644551764,Test Accuracy:3.685252986015245


Mean test accuracy:3.9764476826353246 ,Standard deviation:0.3970435598950846


epoch,▂▄▆█▁▂▇▂▃▃▅▅▇▁█▄▅▅▆█▃▄▅▆▇▅▅▇▇█▂▃▅▆█▁▄▅▇█
fold index,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▅▅▆▆▆▆▆▆▆▆▆▇▇▇▇█████
test_accuracy,▂▃▃▄▆▇▇▇▁▁▇▁▃▃▅▄▄█▂▂▆▇▄▇█▆▇▇▃▅▆██▁▂▆▆▆▆▇
train loss,▃▃▂▇▃▇▆▃▃▁▆▅▄▃▂▂▁▃▂▂▄▂▂▇▆▃▃▁▁▄▂▂▁▁█▅▄▂▇▁
train_accuracy,▁▁▁▆▇▇▂▅▅▆▇█▂▂▅▂▃▃▅▂▆▇▆▇█▂▆▆▂▃▄▄▅▇▂▄█▅▅▇
epoch,49
fold index,10
test_accuracy,3.68525
train loss,5.53354
train_accuracy,3.59103


In [4]:
np.random.seed(42)
input_dim=1024
hidden_dim=512
learning_rate=0.01
num_epochs=5
activation='relu'
batch_size=1

In [5]:
wandb.init(project="SMAI-A2", name="2.1-Relu-SGD")

fold_dirs = [f'Symbols/classification-task/fold-{i}' for i in range(1, 11)]

test_accuracies=[]
for fold_index,fold in enumerate(fold_dirs):
    print("Loading Data")
    train_path = os.path.join(fold, 'train.csv')
    test_path = os.path.join(fold, 'test.csv')
    x_train,y_train=load_data(train_path)
    x_test,y_test=load_data(test_path)
    
    encoder = OneHotEncoder(sparse_output=False)
    y_train = encoder.fit_transform(y_train)
    encoder = OneHotEncoder(sparse_output=False)
    y_test = encoder.fit_transform(y_test)
    
    output_dim=y_train.shape[1]
    
    print("Initialising model")
    model = MLP(input_dim, hidden_dim, output_dim,activation)   
    model.train(x_train,y_train,num_epochs,learning_rate,x_test,y_test,fold_index,batch_size)
    y_pred_test=model.predict(x_test)
    test_accuracy=model.calculate_test_accuracy(y_test,y_pred_test)
    test_accuracies.append(test_accuracy)
    
print(f"Mean test accuracy:{np.mean(test_accuracies)} ,Standard deviation:{np.std(test_accuracies)}")
wandb.finish()

Loading Data
Initialising model


Fold 1/10:  20%|██████████                                        | 1/5 [06:58<27:54, 418.58s/epoch]

Epoch: 0,Loss:3.186585573834487
Train Accuracy:32.505735878498555,Test Accuracy:38.70645009416196


Fold 1/10: 100%|██████████████████████████████████████████████████| 5/5 [36:59<00:00, 443.89s/epoch]


Loading Data
Initialising model


...

Fold 10/10:  20%|█████████▊                                       | 1/5 [07:28<29:52, 448.02s/epoch]

Epoch: 0,Loss:3.222654435110858
Train Accuracy:31.942575145805293,Test Accuracy:36.33035231978873


Fold 10/10: 100%|█████████████████████████████████████████████████| 5/5 [36:52<00:00, 442.44s/epoch]


Mean test accuracy:45.83704585494711 ,Standard deviation:1.4192017495570537


epoch,▁▃▅▆█▃▅▆█▁▅▆█▁▃▆█▁▃▅█▁▃▅▆▁▃▅▆▁▆█▁▃▅█▁▃▅█
fold index,▁▁▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇████
test_accuracy,▃▅▅▇▆▆▅▆▅▃▁▅▇▄▁▅▆▂▅▄▆▂▆█▆▄▅▆▆▃▅▆▄▆▄▇▂▆▇▆
train loss,▇▃▂▂▁▄▃▂▂█▃▂▂█▄▂▂█▄▃▂▇▂▂▁▃▂▂█▄▂▁█▄▃▂█▄▃▁
train_accuracy,▂▅▇▇█▅▆▇▇▁▆▇▇▂▅▇▇▁▅▆█▂▆▇▇▁▅▆▇▁▆█▁▅▆▇▂▅▆█
epoch,4
fold index,10
test_accuracy,45.93962
train loss,2.4552
train_accuracy,46.77051


In [7]:
np.random.seed(42)
input_dim=1024
hidden_dim=512
learning_rate=0.01
num_epochs=30
activation='relu'
batch_size=256

In [8]:
wandb.init(project="SMAI-A2", name="2.1-Relu-MBGD")

fold_dirs = [f'Symbols/classification-task/fold-{i}' for i in range(1, 11)]

test_accuracies=[]
for fold_index,fold in enumerate(fold_dirs):
    print("Loading Data")
    train_path = os.path.join(fold, 'train.csv')
    test_path = os.path.join(fold, 'test.csv')
    x_train,y_train=load_data(train_path)
    x_test,y_test=load_data(test_path)
    
    encoder = OneHotEncoder(sparse_output=False)
    y_train = encoder.fit_transform(y_train)
    encoder = OneHotEncoder(sparse_output=False)
    y_test = encoder.fit_transform(y_test)
    
    output_dim=y_train.shape[1]
    
    print("Initialising model")
    model = MLP(input_dim, hidden_dim, output_dim,activation)   
    model.train(x_train,y_train,num_epochs,learning_rate,x_test,y_test,fold_index,batch_size)
    y_pred_test=model.predict(x_test)
    test_accuracy=model.calculate_test_accuracy(y_test,y_pred_test)
    test_accuracies.append(test_accuracy)
    
print(f"Mean test accuracy:{np.mean(test_accuracies)} ,Standard deviation:{np.std(test_accuracies)}")
wandb.finish()

Loading Data
Initialising model


Fold 1/10:   3%|█▋                                                | 1/30 [00:09<04:35,  9.50s/epoch]

Epoch: 0,Loss:5.090175263497532
Train Accuracy:11.42242467863727,Test Accuracy:18.349811676082865


Fold 1/10:  33%|████████████████▎                                | 10/30 [01:32<03:03,  9.18s/epoch]

Epoch: 9,Loss:2.2331807020096126
Train Accuracy:53.196080721603494,Test Accuracy:53.14854048964218


...

Epoch: 19,Loss:1.7639574809760146
Train Accuracy:60.778219856661046,Test Accuracy:57.709621271232216


Fold 10/10: 100%|████████████████████████████████████████████████| 30/30 [04:38<00:00,  9.28s/epoch]

Epoch: 29,Loss:1.5727943651547778
Train Accuracy:64.40134907251264,Test Accuracy:61.16079467018787


Mean test accuracy:61.90600215594844 ,Standard deviation:0.8109432136584163


epoch,▂▆▆▇▅▆▇▇█▃▇▇▁▂▃▇█▁▂▅▇██▄▆▇▁▁▃▃▁▁▁▄▇▆▆▇▇▇
fold index,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▃▃▃▃▃▃▄▄▅▅▅▆▆▆▆▆▆▇▇▇▇█████
test_accuracy,▆▇▁▇█▇▇█▅▇██▃▆▇████▃▇▇███▃▆█▃▄████▃▅▇███
train loss,▄▂▂▁▁▂▂▁█▆▇▃▂▂▁▃▂▂▂▁▇▄▃▂▂▂▂▄▃▂▁▆▂▁▁█▇▅▄▁
train_accuracy,▃▅▆▇▇████▄█▆█▁▅▇▇███▄▆▇██▅▆▆▇▇█▆▃▆▇█▇▇▄█
epoch,29
fold index,10
test_accuracy,61.16079
train loss,1.57279
train_accuracy,64.40135


In [5]:
np.random.seed(42)
input_dim=1024
hidden_dim=512
learning_rate=0.01
num_epochs=50
activation='sigmoid'
batch_size=None

In [6]:
wandb.init(project="SMAI-A2", name="2.1-Sigmoid-BGD")

fold_dirs = [f'Symbols/classification-task/fold-{i}' for i in range(1, 11)]

test_accuracies=[]
for fold_index,fold in enumerate(fold_dirs):
    print("Loading Data")
    train_path = os.path.join(fold, 'train.csv')
    test_path = os.path.join(fold, 'test.csv')
    x_train,y_train=load_data(train_path)
    x_test,y_test=load_data(test_path)
    
    encoder = OneHotEncoder(sparse_output=False)
    y_train = encoder.fit_transform(y_train)
    encoder = OneHotEncoder(sparse_output=False)
    y_test = encoder.fit_transform(y_test)
    
    output_dim=y_train.shape[1]
    
    print("Initialising model")
    model = MLP(input_dim, hidden_dim, output_dim,activation)   
    model.train(x_train,y_train,num_epochs,learning_rate,x_test,y_test,fold_index,batch_size)
    y_pred_test=model.predict(x_test)
    test_accuracy=model.calculate_test_accuracy(y_test,y_pred_test)
    test_accuracies.append(test_accuracy)
    
print(f"Mean test accuracy:{np.mean(test_accuracies)} ,Standard deviation:{np.std(test_accuracies)}")
wandb.finish()

Loading Data
Initialising model


Fold 1/10:   2%|█                                                 | 1/50 [00:15<12:25, 15.21s/epoch]

Epoch: 0,Loss:6.258077768496321
Train Accuracy:0.30216674050026116,Test Accuracy:0.294256120527307


Fold 1/10:  20%|█████████▊                                       | 10/50 [01:52<06:23,  9.59s/epoch]

Epoch: 9,Loss:6.144501799985322
Train Accuracy:0.3636580027902487,Test Accuracy:0.3825329566854991


...

Epoch: 39,Loss:5.835054960447865
Train Accuracy:2.0887762911355656,Test Accuracy:2.2267570974131203


Fold 10/10: 100%|████████████████████████████████████████████████| 50/50 [07:22<00:00,  8.85s/epoch]

Epoch: 49,Loss:5.774814347057074
Train Accuracy:2.4760509856701765,Test Accuracy:2.568873416961767


Mean test accuracy:2.0930276535834453 ,Standard deviation:0.5130876289100148


epoch,▂▅▅▇▄█▁▂▂▃▇▁▃▃▄█▁█▂▄▅▅▅▆▆▃▃▇█▂▅▅▇██▁▁▂▅█
fold index,▁▁▁▁▂▂▂▃▃▃▃▃▃▃▃▄▅▅▅▆▆▆▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇███
test_accuracy,▁▂▂▄▂▄▆▃▄▄▆▆▃▃▃▅▃▃▃▃▅█▁▂▂▃▁▁▁▁▆▆▆▆▃▄▄▄▃▇
train loss,█▆▆▄▂▆▅▅▄▄▂▅▄▁▆▄▄▂▂▄▅▅▄▄▃▅▅▄▃▁▅▅▃▂█▇▇▂▆▄
train_accuracy,▄▄▄▂▃▄▂▄▄▄▇▇▇▃▃▆▂▂▃▆█▁▁▄▅▁▁▂▄▅▂▆▆▃▄▄▄▁▁▂
epoch,49
fold index,10
test_accuracy,2.56887
train loss,5.77481
train_accuracy,2.47605


In [4]:
np.random.seed(42)
input_dim=1024
hidden_dim=512
learning_rate=0.01
num_epochs=3
activation='sigmoid'
batch_size=1

In [5]:
wandb.init(project="SMAI-A2", name="2.1-Sigmoid-SGD")

fold_dirs = [f'Symbols/classification-task/fold-{i}' for i in range(1, 11)]

test_accuracies=[]
for fold_index,fold in enumerate(fold_dirs):
    print("Loading Data")
    train_path = os.path.join(fold, 'train.csv')
    test_path = os.path.join(fold, 'test.csv')
    x_train,y_train=load_data(train_path)
    x_test,y_test=load_data(test_path)
    
    encoder = OneHotEncoder(sparse_output=False)
    y_train = encoder.fit_transform(y_train)
    encoder = OneHotEncoder(sparse_output=False)
    y_test = encoder.fit_transform(y_test)
    
    output_dim=y_train.shape[1]
    
    print("Initialising model")
    model = MLP(input_dim, hidden_dim, output_dim,activation)   
    model.train(x_train,y_train,num_epochs,learning_rate,x_test,y_test,fold_index,batch_size)
    y_pred_test=model.predict(x_test)
    test_accuracy=model.calculate_test_accuracy(y_test,y_pred_test)
    test_accuracies.append(test_accuracy)
    
print(f"Mean test accuracy:{np.mean(test_accuracies)} ,Standard deviation:{np.std(test_accuracies)}")
wandb.finish()

Loading Data
Initialising model


Fold 1/10:  33%|████████████████▋                                 | 1/3 [05:21<10:43, 321.72s/epoch]

Epoch: 0,Loss:2.3948914570499915
Train Accuracy:48.915307357131994,Test Accuracy:61.4289077212806


Fold 1/10: 100%|██████████████████████████████████████████████████| 3/3 [19:33<00:00, 391.27s/epoch]


Loading Data
Initialising model


...

Fold 10/10:  33%|████████████████▎                                | 1/3 [07:11<14:22, 431.45s/epoch]

Epoch: 0,Loss:2.3796112093009536
Train Accuracy:49.18850447312168,Test Accuracy:60.62061100774263


Fold 10/10: 100%|█████████████████████████████████████████████████| 3/3 [21:27<00:00, 429.31s/epoch]


Mean test accuracy:70.28681640663532 ,Standard deviation:0.3799426231114455


epoch,▁▅█▁▅█▁▅█▁▅█▁▅█▁▅█▁▅█▁▅█▁▅█▁▅█
fold index,▁▁▁▂▂▂▃▃▃▃▃▃▄▄▄▅▅▅▆▆▆▆▆▆▇▇▇███
test_accuracy,▂▅█▂▆█▂▆▇▁▆█▁▅█▁▆▇▂▆▇▂▆▇▂▅█▁▅█
train loss,█▂▁█▂▁█▂▁█▂▁█▂▁█▂▁█▂▁█▂▁█▂▁█▂▁
train_accuracy,▁▆█▁▆█▁▆█▁▆█▁▆█▁▆█▁▆█▁▆█▁▆█▁▆█
epoch,2
fold index,10
test_accuracy,70.42194
train loss,1.17064
train_accuracy,70.11585


In [5]:
np.random.seed(42)
input_dim=1024
hidden_dim=512
learning_rate=0.01
num_epochs=30
activation='sigmoid'
batch_size=256

In [7]:
wandb.init(project="SMAI-A2", name="2.1-Sigmoid-MBGD")

fold_dirs = [f'Symbols/classification-task/fold-{i}' for i in range(1, 11)]

test_accuracies=[]
for fold_index,fold in enumerate(fold_dirs):
    print("Loading Data")
    train_path = os.path.join(fold, 'train.csv')
    test_path = os.path.join(fold, 'test.csv')
    x_train,y_train=load_data(train_path)
    x_test,y_test=load_data(test_path)
    
    encoder = OneHotEncoder(sparse_output=False)
    y_train = encoder.fit_transform(y_train)
    encoder = OneHotEncoder(sparse_output=False)
    y_test = encoder.fit_transform(y_test)
    
    output_dim=y_train.shape[1]
    
    print("Initialising model")
    model = MLP(input_dim, hidden_dim, output_dim,activation)   
    model.train(x_train,y_train,num_epochs,learning_rate,x_test,y_test,fold_index,batch_size)
    y_pred_test=model.predict(x_test)
    test_accuracy=model.calculate_test_accuracy(y_test,y_pred_test)
    test_accuracies.append(test_accuracy)
    
print(f"Mean test accuracy:{np.mean(test_accuracies)} ,Standard deviation:{np.std(test_accuracies)}")
wandb.finish()

Loading Data
Initialising model


Fold 1/10:   3%|█▋                                                | 1/30 [00:09<04:48,  9.96s/epoch]

Epoch: 0,Loss:5.456389487491301
Train Accuracy:4.343391834187775,Test Accuracy:6.455979284369115


Fold 1/10:  33%|████████████████▎                                | 10/30 [01:36<03:15,  9.77s/epoch]

Epoch: 9,Loss:4.25527078899339
Train Accuracy:23.640446377461256,Test Accuracy:24.017184557438796


...

Epoch: 19,Loss:3.398588467417247
Train Accuracy:37.828309443507585,Test Accuracy:37.71082167937099


Fold 10/10: 100%|████████████████████████████████████████████████| 30/30 [04:30<00:00,  9.00s/epoch]

Epoch: 29,Loss:2.881095668517601
Train Accuracy:44.940319350758855,Test Accuracy:44.77522357601585


Mean test accuracy:44.82692649435957 ,Standard deviation:0.209043618358431


epoch,▁▃▅▅▅▆▇█▁▄▇█▁▃▄▆▁▂█▆▃█▁▃▃▅█▄▇▇█▁▂▅▇▂█▄▅▇
fold index,▁▁▁▁▁▁▁▁▂▂▃▃▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇█████
test_accuracy,▂▃▄▆▂▇▁▄▅█▇█▃▄▄▅▇▂▃▆▄▅▇█▄▆▆▇██▃▄▆██▇█▂▅▇
train loss,▆▄▄▁█▆▃▅▄▂▇▅▄▂▁▇▆▅▄▃▇▅▄▃▂▇▄▁▆▄▂▆▆▄▂▇▅▅▃▁
train_accuracy,▆▇▃▄▄▁▄▆▁▄▇▄▅▆▆▂▃▆▆▇█▆▇██▅▇▁▂▅█▁███▂▃▆▆█
epoch,29
fold index,10
test_accuracy,44.77522
train loss,2.8811
train_accuracy,44.94032


In [7]:
np.random.seed(42)
input_dim=1024
hidden_dim=512
learning_rate=0.01
num_epochs=50
activation='tanh'
batch_size=None

In [8]:
wandb.init(project="SMAI-A2", name="2.1-Tanh-BGD")

fold_dirs = [f'Symbols/classification-task/fold-{i}' for i in range(1, 11)]

test_accuracies=[]
for fold_index,fold in enumerate(fold_dirs):
    print("Loading Data")
    train_path = os.path.join(fold, 'train.csv')
    test_path = os.path.join(fold, 'test.csv')
    x_train,y_train=load_data(train_path)
    x_test,y_test=load_data(test_path)
    
    encoder = OneHotEncoder(sparse_output=False)
    y_train = encoder.fit_transform(y_train)
    encoder = OneHotEncoder(sparse_output=False)
    y_test = encoder.fit_transform(y_test)
    
    output_dim=y_train.shape[1]
    
    print("Initialising model")
    model = MLP(input_dim, hidden_dim, output_dim,activation)   
    model.train(x_train,y_train,num_epochs,learning_rate,x_test,y_test,fold_index,batch_size)
    y_pred_test=model.predict(x_test)
    test_accuracy=model.calculate_test_accuracy(y_test,y_pred_test)
    test_accuracies.append(test_accuracy)
    
print(f"Mean test accuracy:{np.mean(test_accuracies)} ,Standard deviation:{np.std(test_accuracies)}")
wandb.finish()

Loading Data
Initialising model


Fold 1/10:   2%|█                                                 | 1/50 [00:07<06:20,  7.77s/epoch]

Epoch: 0,Loss:6.452403107465139
Train Accuracy:0.37886551927056816,Test Accuracy:0.3295668549905838


Fold 1/10:  20%|█████████▊                                       | 10/50 [01:20<05:21,  8.04s/epoch]

Epoch: 9,Loss:6.10298970371785
Train Accuracy:0.3643191991589582,Test Accuracy:0.3472222222222222


...

Epoch: 39,Loss:5.615317698651193
Train Accuracy:3.0652099332330507,Test Accuracy:2.9830142248364444


Fold 10/10: 100%|████████████████████████████████████████████████| 50/50 [07:07<00:00,  8.56s/epoch]

Epoch: 49,Loss:5.551788426196373
Train Accuracy:3.581796110099491,Test Accuracy:3.4691795210371525


Mean test accuracy:3.741727654366464 ,Standard deviation:0.5051265951542983


epoch,▁▃▃▇█▂▃▃▄▅██▁▁▂▃▅▆▇▅▆▅█▃▃▂▄▅▅▆██▁▁▂█▃▃▅█
fold index,▁▁▁▁▁▂▂▃▃▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇██
test_accuracy,▁▄▅▆▆▆▇▇▃▆█▁▂▃▅▅▅▆▂▇▃▃▄▇▆▆▇▁▁▆██▁▇█▁▃▆▆▇
train loss,▆▅▃▃▂▃▂▁▃▃▁▇▇▅▄▂▁▆▅▂▆▆▅▄▃▂▆▃▂▂▃▂█▅▅▂▄▄▄▂
train_accuracy,▁▁▂▃▅▆▆▃▆▆▇▇▇▃▄▄▅▅▆█▅▆▇▇▇▅▆▆▃▆▁▁▂▂▃▇██▁▅
epoch,49
fold index,10
test_accuracy,3.46918
train loss,5.55179
train_accuracy,3.5818


In [4]:
np.random.seed(42)
input_dim=1024
hidden_dim=512
learning_rate=0.01
num_epochs=3
activation='tanh'
batch_size=1

In [5]:
wandb.init(project="SMAI-A2", name="2.1-Tanh-SGD")

fold_dirs = [f'Symbols/classification-task/fold-{i}' for i in range(1, 11)]

test_accuracies=[]
for fold_index,fold in enumerate(fold_dirs):
    print("Loading Data")
    train_path = os.path.join(fold, 'train.csv')
    test_path = os.path.join(fold, 'test.csv')
    x_train,y_train=load_data(train_path)
    x_test,y_test=load_data(test_path)
    
    encoder = OneHotEncoder(sparse_output=False)
    y_train = encoder.fit_transform(y_train)
    encoder = OneHotEncoder(sparse_output=False)
    y_test = encoder.fit_transform(y_test)
    
    output_dim=y_train.shape[1]
    
    print("Initialising model")
    model = MLP(input_dim, hidden_dim, output_dim,activation)   
    model.train(x_train,y_train,num_epochs,learning_rate,x_test,y_test,fold_index,batch_size)
    y_pred_test=model.predict(x_test)
    test_accuracy=model.calculate_test_accuracy(y_test,y_pred_test)
    test_accuracies.append(test_accuracy)
    
print(f"Mean test accuracy:{np.mean(test_accuracies)} ,Standard deviation:{np.std(test_accuracies)}")
wandb.finish()

Loading Data
Initialising model


Fold 1/10:  33%|████████████████▋                                 | 1/3 [06:23<12:47, 383.70s/epoch]

Epoch: 0,Loss:3.0902145099197793
Train Accuracy:35.4811195376915,Test Accuracy:41.17231638418079


Fold 1/10: 100%|██████████████████████████████████████████████████| 3/3 [17:28<00:00, 349.57s/epoch]


Loading Data
Initialising model


...

Fold 10/10:  33%|████████████████▎                                | 1/3 [06:30<13:01, 390.72s/epoch]

Epoch: 0,Loss:3.0747299135133894
Train Accuracy:35.806085556699124,Test Accuracy:38.46107676610047


Fold 10/10: 100%|█████████████████████████████████████████████████| 3/3 [23:46<00:00, 475.40s/epoch]


Mean test accuracy:36.88583153361808 ,Standard deviation:1.642102474282666


epoch,▁▅█▁▅█▁▅█▁▅█▁▅█▁▅█▁▅█▁▅█▁▅█▁▅█
fold index,▁▁▁▂▂▂▃▃▃▃▃▃▄▄▄▅▅▅▆▆▆▆▆▆▇▇▇███
test_accuracy,█▅▄▆▇▅▇▆▆▇█▆▃▄▆▇█▅▅▁▄▂▆▆█▄▆▆▅▆
train loss,▇▄█▇▂▇▆▃█▆▁▆▆▃▇▆▁▅▇▃▇▇▁▆▆▁▅▆▂▇
train_accuracy,▁▆▅▁▆▅▂▇▄▂▇▆▂▅▅▂█▇▂▆▅▁█▆▂█▇▂█▅
epoch,2
fold index,10
test_accuracy,37.70482
train loss,3.08315
train_accuracy,37.055


In [9]:
np.random.seed(42)
input_dim=1024
hidden_dim=512
learning_rate=0.01
num_epochs=30
activation='tanh'
batch_size=256

In [10]:
wandb.init(project="SMAI-A2", name="2.1-Tanh-MBGD")

fold_dirs = [f'Symbols/classification-task/fold-{i}' for i in range(1, 11)]

test_accuracies=[]
for fold_index,fold in enumerate(fold_dirs):
    print("Loading Data")
    train_path = os.path.join(fold, 'train.csv')
    test_path = os.path.join(fold, 'test.csv')
    x_train,y_train=load_data(train_path)
    x_test,y_test=load_data(test_path)
    
    encoder = OneHotEncoder(sparse_output=False)
    y_train = encoder.fit_transform(y_train)
    encoder = OneHotEncoder(sparse_output=False)
    y_test = encoder.fit_transform(y_test)
    
    output_dim=y_train.shape[1]
    
    print("Initialising model")
    model = MLP(input_dim, hidden_dim, output_dim,activation)   
    model.train(x_train,y_train,num_epochs,learning_rate,x_test,y_test,fold_index,batch_size)
    y_pred_test=model.predict(x_test)
    test_accuracy=model.calculate_test_accuracy(y_test,y_pred_test)
    test_accuracies.append(test_accuracy)
    
print(f"Mean test accuracy:{np.mean(test_accuracies)} ,Standard deviation:{np.std(test_accuracies)}")
wandb.finish()

Loading Data
Initialising model


Fold 1/10:   3%|█▋                                                | 1/30 [00:11<05:23, 11.14s/epoch]

Epoch: 0,Loss:5.131595144341551
Train Accuracy:10.402388701585137,Test Accuracy:17.02565913370998


Fold 1/10:  33%|████████████████▎                                | 10/30 [01:54<03:48, 11.42s/epoch]

Epoch: 9,Loss:2.4737244537414367
Train Accuracy:50.19689912325008,Test Accuracy:49.78813559322034


...

Epoch: 19,Loss:1.9337628419475557
Train Accuracy:58.12908410623946,Test Accuracy:56.785307004381494


Fold 10/10: 100%|████████████████████████████████████████████████| 30/30 [06:02<00:00, 12.08s/epoch]

Epoch: 29,Loss:1.6930920321254068
Train Accuracy:62.24085687183811,Test Accuracy:59.936378368645336


Mean test accuracy:60.643855322149555 ,Standard deviation:0.37521297258197034


epoch,▅▅█▁▄▅▆▁▃▇█▂▃▅▇▇▁▂▃▃▆▇▇█▁▃▅▅▁▄▆▇▃▃▃▅▇▇▃▇
fold index,▁▁▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▆▆▆▆▆▆▆▆▆▆▆▆▇▇▇██████
test_accuracy,▁▆▆▇▇█▁▃▅██▅▆▇▇▇██▆▇▆▆▇█▁▅▆▆▇▇█▄▇█▆▁▄▄▅█
train loss,▅▁▁▆▅▃█▅▅▄▁▁▁▇▅▁▁▅▃▂▂▁▁▄▂▁▄▂▁▁▃▂▂▂▁▄▂▂▁▁
train_accuracy,▃▄▁▄▅▇▇██▅▆▇██▇▇▇██▅▇█▁▆▇███▄▆▇█▅▇▇██▅▆▇
epoch,29
fold index,10
test_accuracy,59.93638
train loss,1.69309
train_accuracy,62.24086


#### Test Accuracy and Standard Deviation for Different Optimizers and Activations

| Activation | Optimizer | Mean Test Accuracy (%) | Standard Deviation |
|------------|-----------|------------------------|---------------------|
| ReLU      | BGD       | 3.98                   | 0.40                |
|           | SGD       | 45.84                  | 1.42                |
|           | MBGD      | 61.91                  | 0.81                |
| Sigmoid   | BGD       | 2.09                   | 0.51                |
|           | SGD       | 36.89                  | 1.64                |
|           | MBGD      | 44.83                  | 0.21                |
| Tanh      | BGD       | 3.74                   | 0.51                |
|           | SGD       | 70.29                  | 0.38                |
|           | MBGD      | 60.64                  | 0.38                |

#### Training Hyperparameters

| Optimizer | Seed | Input Dim | Hidden Dim | Learning Rate | Epochs | Batch Size |
|-----------|------|-----------|------------|---------------|--------|------------|
| BGD       | 42   | 1024      | 512        | 0.01          | 50     | None       |
| MBGD      | 42   | 1024      | 512        | 0.01          | 30     | 256        |
| SGD       | 42   | 1024      | 512        | 0.01          | 10     | 1          |


#### Configurations

1. Optimizer Impact on Accuracy
   - Batch Gradient Descent (BGD) performed the worst across all activations. This is because it updates weights only after processing the entire dataset, leading to slow convergence and suboptimal results.
   - Stochastic Gradient Descent (SGD) significantly improved accuracy compared to BGD. SGD updates weights after each data point, allowing faster learning but also higher variance.
   - Mini-Batch Gradient Descent (MBGD) achieved the best accuracy, especially for ReLU (61.91%) and Tanh (60.64%), as it balances between convergence speed and stability by updating weights in mini-batches.

2. Activation Function Comparison
   - ReLU performed the best overall, especially with MBGD (61.91%), making it the most effective activation.
   - Tanh was close to ReLU, showing good performance (60.64%) in MBGD but poor in BGD.
   - Sigmoid performed the worst, with slightly low accuracy in MBGD (44.83%) . This may be because it suffers from vanishing gradients.

3. 
   - MBGD had the lowest variance across all cases, proving its stability.
   - SGD had the highest variance, suggesting instability due to frequent updates.

- ReLU is the best activation, followed by Tanh, while Sigmoid struggles with deeper networks.
- MBGD is the best optimizer, balancing performance and stability.
- BGD is ineffective for deep networks due to slow learning.
- SGD works well but has high variance, making training less stable.

### Report

1. What do the mean and standard deviation tell you about model
performance and consistency?

Mean tell us the average performance of the model on the test sets across all folds. A higher mean accuracy indicates better generalization to unseen data.
standard deviation reflects the consistency of the model’s performance across different folds. Lower standard deviation means more stable training.

2. How does a high vs. low standard deviation impact confidence in the model’s generalization?

A low standard deviation suggests that the model performs consistently across different subsets of the data, meaning it is more stable and reliable. The accuracy remains constant across all folds on the test data.
A high standard deviation indicates that the model's performance varies significantly across folds, which could imply sensitivity to different training data distributions or possiblly overfitting.

3. If one configuration has a slightly higher mean accuracy but a significantly higher standard deviation compared to another with marginally lower mean accuracy, which would you choose and why?

I would prefer the configuration with the lower standard deviation, even if its mean accuracy is slightly lower. This will ensure better generalisation even on unseen data as a lower standard deviation means that the model's performance is more stable across different folds.

![Alt Text](Graphs\2.1_BGD_train_loss.png)
![Alt Text](Graphs\2.1_BGD_train_acc.png)
![Alt Text](Graphs\2.1_BGD_test_acc.png)
![Alt Text](Graphs\2.1._BGD_fold_idx.png)

![Alt Text](Graphs\2.1_MBGD_train_loss.png)
![Alt Text](Graphs\2.1_MBGD_train_acc.png)
![Alt Text](Graphs\2.1_MBGD_test_acc.png)
![Alt Text](Graphs\2.1_MBGD_fold_idx.png)

![Alt Text](Graphs\2.1_SGD_train_loss.png)
![Alt Text](Graphs\2.1_SGD_train_acc.png)
![Alt Text](Graphs\2.1_SGD_test_acc.png)
![Alt Text](Graphs\2.1_SGD_fold_idx.png)